<a href="https://colab.research.google.com/github/OlgaNezh/PyTorch/blob/main/HW_9_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Урок 9. Трансформер

Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
Сделайте предсказания на всем df_val. Посчитайте метрику качества.
Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.
Данные на google drive: https://drive.google.com/file/d/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901



In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.0 MB/s 
     |████████████████████████████████| 596 kB 48.3 MB/s 
     |████████████████████████████████| 6.6 MB 48.5 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 361 kB 6.8 MB/s 
     |████████████████████████████████| 212 kB 58.3 MB/s 
     |████████████████████████████████| 1.1 MB 69.5 MB/s 
     |████████████████████████████████| 140 kB 72.9 MB/s 
     |████████████████████████████████| 127 kB 69.2 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |████████████████████████████████| 144 kB 70.3 MB/s 
     |███████████████

In [2]:
import numpy as np

import torch
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm

from transformers import BertTokenizer, BertModel
from datasets import load_dataset

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL = 'cross-encoder/ms-marco-MiniLM-L-12-v2'
EPOCHS = 10
BATCH_SIZE = 128
MAX_LENGTH = 30

emotion_dataset = load_dataset("emotion")

Using custom data configuration default


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [5]:
bertTokenizer = BertTokenizer.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/791 [00:00<?, ?B/s]

In [6]:
example_text = emotion_dataset["train"]["text"][0]
bert_input = bertTokenizer(example_text, padding='max_length', max_length=MAX_LENGTH, truncation=True, return_tensors="pt")
print(example_text)
print(bert_input)

i didnt feel humiliated
{'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]])}


In [7]:
bertTokenizer.ids_to_tokens[26608], bertTokenizer.ids_to_tokens[1045]

('humiliated', 'i')

In [10]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained(MODEL)
        self._txts = [self.tokenizer(text, padding='max_length', max_length=MAX_LENGTH,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [11]:
train_dataset = TwitterDataset(emotion_dataset["train"]['text'], emotion_dataset["train"]["label"])
valid_dataset = TwitterDataset(emotion_dataset["test"]['text'],  emotion_dataset["test"]["label"])

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=False,
                          num_workers=1)

In [12]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.bert = BertModel.from_pretrained(MODEL)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(384, 64)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        
        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)
        return final_layer

In [13]:
model = BertClassifier().to(DEVICE)
criterion = nn.CrossEntropyLoss()

optimizer = Adam(model.linear.parameters(), lr=1e-5)

Downloading:   0%|          | 0.00/127M [00:00<?, ?B/s]

Some weights of the model checkpoint at cross-encoder/ms-marco-MiniLM-L-12-v2 were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.linear.parameters()]))

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=Tru

In [15]:
for epoch_num in range(EPOCHS):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(DEVICE)
        input_id = train_input['input_ids'].squeeze(1).to(DEVICE)
        train_label = train_label.to(DEVICE)

        output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(DEVICE)
        mask = val_input['attention_mask'].to(DEVICE)
        input_id = val_input['input_ids'].squeeze(1).to(DEVICE)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

100%|██████████| 125/125 [00:22<00:00,  5.65it/s]


Epochs: 1 | Train Loss:  0.032         | Train Accuracy:  0.043         | Val Loss:  0.032         | Val Accuracy:  0.237


100%|██████████| 125/125 [00:22<00:00,  5.51it/s]


Epochs: 2 | Train Loss:  0.031         | Train Accuracy:  0.175         | Val Loss:  0.031         | Val Accuracy:  0.289


100%|██████████| 125/125 [00:23<00:00,  5.33it/s]


...

Epochs: 8 | Train Loss:  0.028         | Train Accuracy:  0.274         | Val Loss:  0.028         | Val Accuracy:  0.290


100%|██████████| 125/125 [00:23<00:00,  5.24it/s]


Epochs: 9 | Train Loss:  0.027         | Train Accuracy:  0.279         | Val Loss:  0.028         | Val Accuracy:  0.290


100%|██████████| 125/125 [00:23<00:00,  5.23it/s]


Epochs: 10 | Train Loss:  0.027         | Train Accuracy:  0.279         | Val Loss:  0.028         | Val Accuracy:  0.290
